## Homework-5

In [1]:
import requests
import datetime
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric, ColumnQuantileMetric

from joblib import load, dump
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [21]:
mar24_data = pd.read_parquet('data/green_tripdata_2024-03.parquet')
mar24_data.shape

(57457, 20)

In [22]:
mar24_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2024-03-01 00:10:52,2024-03-01 00:26:12,N,1.0,129,226,1.0,1.72,12.8,1.0,0.5,3.06,0.0,NaN,1.0,18.36,1.0,1.0,0.00
1,2,2024-03-01 00:22:21,2024-03-01 00:35:15,N,1.0,130,218,1.0,3.25,17.7,1.0,0.5,0.00,0.0,NaN,1.0,20.20,2.0,1.0,0.00
2,2,2024-03-01 00:45:27,2024-03-01 01:04:32,N,1.0,255,107,2.0,4.58,23.3,1.0,0.5,3.50,0.0,NaN,1.0,32.05,1.0,1.0,2.75
3,1,2024-03-01 00:02:00,2024-03-01 00:23:45,N,1.0,181,71,1.0,0.00,22.5,0.0,1.5,0.00,0.0,NaN,1.0,24.00,1.0,1.0,0.00
4,2,2024-03-01 00:16:45,2024-03-01 00:23:25,N,1.0,95,135,1.0,1.15,8.6,1.0,0.5,1.00,0.0,NaN,1.0,12.10,1.0,1.0,0.00


In [23]:
mar24_data["lpep_pickup_datetime"].unique

<bound method Series.unique of 0       2024-03-01 00:10:52
1       2024-03-01 00:22:21
2       2024-03-01 00:45:27
3       2024-03-01 00:02:00
4       2024-03-01 00:16:45
                ...        
57452   2024-03-31 21:19:00
57453   2024-03-31 22:30:00
57454   2024-03-31 22:43:00
57455   2024-03-31 22:48:00
57456   2024-03-31 22:08:00
Name: lpep_pickup_datetime, Length: 57457, dtype: datetime64[us]>

In [3]:
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]

In [4]:
column_mapping = ColumnMapping(
    target=None,
    numerical_features=num_features,
    categorical_features=cat_features
)

In [15]:
mar24_data..unique()

DatetimeIndex(['2024-03-01 00:10:52', '2024-03-01 00:22:21',
               '2024-03-01 00:45:27', '2024-03-01 00:02:00',
               '2024-03-01 00:16:45', '2024-03-01 00:41:20',
               '2024-03-01 00:47:47', '2024-03-01 00:44:48',
               '2024-03-01 00:32:39', '2024-03-01 00:07:41',
               ...
               '2024-03-31 19:47:00', '2024-03-31 20:11:09',
               '2024-03-31 20:56:00', '2024-03-31 20:10:00',
               '2024-03-31 21:29:29', '2024-03-31 21:19:00',
               '2024-03-31 22:30:00', '2024-03-31 22:43:00',
               '2024-03-31 22:48:00', '2024-03-31 22:08:00'],
              dtype='datetime64[us]', name='lpep_pickup_datetime', length=56279, freq=None)

### Finding the max quantile using pandas

In [10]:
mar24_data.set_index('lpep_pickup_datetime', inplace=True)

In [11]:
daily_quantile = mar24_data["fare_amount"].resample('D').quantile(0.5)

In [20]:
daily_quantile[daily_quantile==75]

lpep_pickup_datetime
2024-02-24    75.0
Freq: D, Name: fare_amount, dtype: float64

In [17]:
max(daily_quantile)

75.0